In [1]:
!pip install researchpy

In [2]:
!pip install lifetimes

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno
from datetime import date
import researchpy as rp
from sklearn.preprocessing import MinMaxScaler
#pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

from lifetimes import BetaGeoFitter
from lifetimes.plotting import plot_period_transactions
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler
import datetime as dt

In [2]:
df=pd.read_csv('/Users/serhandulger/flo_data_20k.csv')

In [3]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### NA SUM #####################")
    print(dataframe.isnull().sum().sum())
    print("##################### Describe #####################")
    print(dataframe.describe())
    print("##################### Nunique #####################")
    print(dataframe.nunique())

In [4]:
check_df(df)

In [5]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

In [6]:
missing_values_analysis(df)

In [7]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [8]:
df["Total_Order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["Total_Payment"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [9]:
df.describe().T

In [10]:
extension = ["Total_Order","Total_Payment"]

In [11]:
num_cols = [col for col in df.columns if df[col].dtype != 'O' and col not in extension ]

In [12]:
num_cols

In [13]:
features = ["customer_value_total_ever_online","customer_value_total_ever_offline","order_num_total_ever_offline","order_num_total_ever_online"]

In [14]:
for i in features:
    replace_with_thresholds(df,i)

In [15]:
df.describe().T

In [16]:
import datetime as dt
df["first_order_date"] = pd.to_datetime(df["first_order_date"]).dt.normalize()
df["last_order_date"] = pd.to_datetime(df["last_order_date"]).dt.normalize()
df["last_order_date_online"] = pd.to_datetime(df["last_order_date_online"]).dt.normalize()
df["last_order_date_offline"] = pd.to_datetime(df["last_order_date_offline"]).dt.normalize()

In [17]:
df.shape

In [18]:
df = df[~(df["Total_Payment"] == 0) | (df["Total_Order"] == 0)]

In [19]:
df.shape

In [20]:
df["last_order_date"].max()

In [21]:
today_date = dt.datetime(2021,7,1)

In [22]:
new_df = pd.DataFrame({"CUSTOMER_ID": df["master_id"],
             "RECENCY_WEEKLY": ((df["last_order_date"] - df["first_order_date"]).dt.days)/7,
             "TENURE_WEEKLY": ((today_date - df["first_order_date"]).astype('timedelta64[D]'))/7,
             "FREQUENCY": df["Total_Order"],
             "MONETARY_AVG": df["Total_Payment"] / df["Total_Order"]})

In [23]:
new_df

In [24]:
##############################
# Forecasting expected purchases from customers in 3 months
##############################

In [25]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [26]:
bgf.fit(new_df['FREQUENCY'],
        new_df['RECENCY_WEEKLY'],
        new_df['TENURE_WEEKLY'])

In [27]:
new_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                               new_df['FREQUENCY'],
                                               new_df['RECENCY_WEEKLY'],
                                               new_df['TENURE_WEEKLY'])

In [28]:
new_df.sort_values(by="exp_sales_3_month",ascending=False)[0:10]

In [29]:
##############################
# Forecasting expected purchases from customers in 6 months
##############################

In [30]:
new_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                               new_df['FREQUENCY'],
                                               new_df['RECENCY_WEEKLY'],
                                               new_df['TENURE_WEEKLY'])

In [31]:
new_df.head()

In [32]:
# prediction validation
plot_period_transactions(bgf)
plt.show()

In [33]:
##############################
# GAMMA-GAMMA MODEL 
##############################

In [34]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [35]:
ggf.fit(new_df['FREQUENCY'], new_df['MONETARY_AVG'])

In [36]:
new_df['EXP_AVERAGE_VALUE'] = ggf.conditional_expected_average_profit(new_df['FREQUENCY'], new_df['MONETARY_AVG'])

In [37]:
new_df.head()

In [38]:
##############################
# Calculation of CLTV with BG-NBD and GG model  - (6 MONTHS)
##############################

In [39]:
cltv = ggf.customer_lifetime_value(bgf,
                                   new_df['FREQUENCY'],
                                   new_df['RECENCY_WEEKLY'],
                                   new_df['TENURE_WEEKLY'],
                                   new_df['MONETARY_AVG'],
                                   time=6, # 6 MONTH
                                   freq="W",  # T's frequency information. (WEEKLY)
                                   discount_rate=0.01) # consider discounts that can be made over time (discount rate)

In [40]:
cltv = pd.DataFrame(cltv)

In [41]:
new_df["CLTV"] = cltv

In [42]:
new_df.sort_values(by='CLTV', ascending = False).head()

In [43]:
#########################################
# CREATING SEGMENTS BASED ON CLTV VALUES
#########################################

In [44]:
new_df["SEGMENT"] = pd.qcut(new_df["CLTV"], 4, labels=["D", "C", "B", "A"])

In [45]:
new_df.groupby("SEGMENT").agg({"count","mean","sum"})

In [46]:
new_df.head()